In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
from sklearn.metrics import f1_score 
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Dropout, Conv2D
from PIL import Image
import os
import re
import csv
import cv2
import math
import numpy as np

In [7]:
# USING LAPLACIAN VARIANCE

def variance_of_laplacian(image):
    # Compute the Laplacian of the image and then return the focus
    # measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(image, cv2.CV_64F).var()

def detect_blur(image, threshold=100):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Compute the variance of Laplacian
    blur = variance_of_laplacian(gray)
    # The image is considered blurry if the variance of Laplacian is below a threshold
    return blur < threshold

def calculate_average_blur(folder_path, threshold=100):
    total_blur = 0
    num_images = 0

    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            # Load the image
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)

            # Detect blur
            is_blurry = detect_blur(image, threshold)

            # If the image is not blurry, calculate blur value and add to total
            if not is_blurry:
                blur_value = variance_of_laplacian(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
                total_blur += blur_value
                num_images += 1

    # Calculate average blur value
    if num_images > 0:
        average_blur = total_blur / num_images
        print(f"Average blur value: {average_blur}")
    else:
        print("No images found or all images are blurry.")

# Specify the folder containing images
folder_path = "hazy_image"

# Calculate average blur value for images in the folder
calculate_average_blur(folder_path)


Average blur value: 5159.081676035668


In [8]:
# USING DARK CHANNEL PRIOR

def dark_channel_prior(image, window_size=15):
    # Convert the image to float
    img_float = image.astype(np.float64) / 255.0

    # Calculate the dark channel
    b, g, r = cv2.split(img_float)
    dark_channel = np.minimum(np.minimum(r, g), b)

    # Use a local window to find the minimum value in the dark channel
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (window_size, window_size))
    dark_channel_min = cv2.erode(dark_channel, kernel)

    return dark_channel_min

def haze_degree(image):
    # Compute the dark channel prior
    dark_channel_min = dark_channel_prior(image)

    # Estimate the atmospheric light
    atmospheric_light = np.percentile(dark_channel_min, 95)

    # Calculate the haze degree
    haze_degree = 1.0 - dark_channel_min / atmospheric_light

    # Average the haze degree over the image
    average_haze_degree = np.mean(haze_degree)

    return average_haze_degree


def calculate_average_haze_degree(folder_path):
    total_haze_score = 0
    num_images = 0

    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            # Load the image
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)

            # Calculate haze degree
            haze_score = haze_degree(image)

            total_haze_score += haze_score
            num_images += 1

            print(f"{filename}: Haze degree: {haze_score}")

    # Calculate average haze degree
    if num_images > 0:
        average_haze_score = total_haze_score / num_images
        print(f"Average haze degree for all images: {average_haze_score}")
    else:
        print("No images found in the folder.")

# Specify the folder containing images
folder_path = "hazy_image"

# Calculate average haze degree for images in the folder
calculate_average_haze_degree(folder_path)


10.png: Haze degree: 0.3939003097684442
101.png: Haze degree: 0.32393778286013375
103.png: Haze degree: 0.3622136978975544
110.png: Haze degree: 0.39749981616928975
112.png: Haze degree: 0.3479536500513451
116.png: Haze degree: 0.3488735944109256
119.png: Haze degree: 0.3107030149938983
120.png: Haze degree: 0.32722901608053634
121.png: Haze degree: 0.3299285720381291
124.png: Haze degree: 0.30285060433574273
129.png: Haze degree: 0.34000674170727335
136.png: Haze degree: 0.3402496707372925
144.png: Haze degree: 0.38654269890169907
15.png: Haze degree: 0.36909209549122296
151.png: Haze degree: 0.37062208606059566
155.png: Haze degree: 0.32545833653351086
156.png: Haze degree: 0.3521057167970917
157.png: Haze degree: 0.3436059600105819
158.png: Haze degree: 0.3106012066802639
159.png: Haze degree: 0.341768802109557
160.png: Haze degree: 0.3296130203133045
173.png: Haze degree: 0.3202359028625233
174.png: Haze degree: 0.325482999234802
195.png: Haze degree: 0.3470217868084416
196.png: Ha

In [9]:
# USING COLOR ATTENUATION PRIOR

def haze_detection_CAP(image):
    # Convert the image to float
    img_float = image.astype(np.float64) / 255.0

    # Calculate the dark channel prior
    b, g, r = cv2.split(img_float)
    dark_channel = np.minimum(np.minimum(r, g), b)

    # Estimate the atmospheric light
    atmospheric_light = np.percentile(dark_channel, 95)

    # Estimate the transmission map
    transmission_map = 1 - 0.95 * dark_channel / atmospheric_light

    # Calculate the haze density
    haze_density = 1 - transmission_map

    return haze_density

def calculate_average_haze_density(folder_path):
    total_haze_density = 0
    num_images = 0

    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            # Load the image
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)

            # Calculate haze density
            haze_density_map = haze_detection_CAP(image)

            total_haze_density += np.mean(haze_density_map)
            num_images += 1

            print(f"{filename}: Mean Haze Density: {np.mean(haze_density_map)}")

    # Calculate average haze density
    if num_images > 0:
        average_haze_density = total_haze_density / num_images
        print(f"Average haze density for all images: {average_haze_density}")
    else:
        print("No images found in the folder.")

# Specify the folder containing images
folder_path = "hazy_image"

# Calculate average haze density for images in the folder
calculate_average_haze_density(folder_path)


10.png: Mean Haze Density: 0.7635075233199481
101.png: Mean Haze Density: 0.8197339087963633
103.png: Mean Haze Density: 0.7790608033725602
110.png: Mean Haze Density: 0.7618877949046723
112.png: Mean Haze Density: 0.7951089482652192
116.png: Mean Haze Density: 0.7873180094611468
119.png: Mean Haze Density: 0.8217674167882735
120.png: Mean Haze Density: 0.8092035213330944
121.png: Mean Haze Density: 0.8101396500570341
124.png: Mean Haze Density: 0.8348477355543985
129.png: Mean Haze Density: 0.7982395652878879
136.png: Mean Haze Density: 0.794061076454374
144.png: Mean Haze Density: 0.756227785790074
15.png: Mean Haze Density: 0.7779826306055244
151.png: Mean Haze Density: 0.7764592297386095
155.png: Mean Haze Density: 0.8171316865931051
156.png: Mean Haze Density: 0.7942805631174564
157.png: Mean Haze Density: 0.7952005586798768
158.png: Mean Haze Density: 0.830227003524462
159.png: Mean Haze Density: 0.7992059335557812
160.png: Mean Haze Density: 0.8105288997363054
173.png: Mean Haze